# Building a Spam Filter with Naive Bayes
In this project, our goal is to build a spam filter using Naive Bayes that with an accuracy of at least 80% can classify messages as "spam" or "non-spam".

To classify messages as spam or non-spam a computer:
1. Learns how humans classify messages.
2. Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
3. Classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).

Our first task is to "teach" the computer how to classify messages. To do that, we'll use the __multinomial Naive Bayes__ algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the authors' papers.

Let's read in the file using pandas. It is tab-seperated and it includes no headers, so we'll account for this.

In [1]:
import pandas as pd
spam_sms = pd.read_csv('SMSSpamCollection', sep = '\t', header = None, names=['Label', 'SMS'])

In [2]:
spam_sms.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


`ham` corresponds to non-spam and the meaning of `spam` is quite self-explanatorily the opposite. Let's determine what percentage of the collection is spam:

In [3]:
spam_sms['Label'].value_counts(normalize=True) * 100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

13.4% of the messages in the collection are marked as spam, and the remaining 86.6% as non-spam.

## Training and test set
Once our spam filter is done, we'll need to test how good it is with classifying new messages. To test the spam filter, we're first going to split our dataset into two categories:
- A training set, which we'll use to "train" the computer how to classify messages.
- A test set, which we'll use to test how good the spam filter is with classifying new messages.

We're going to keep 80% of our dataset for training, and 20% for testing (we want to train the algorithm on as much data as possible, but we also want to have enough test data). The dataset has 5,572 messages, which means that:
- The training set will have 4,458 messages (about 80% of the dataset).
- The test set will have 1,114 messages (about 20% of the dataset).

For this project, our goal is to create a spam filter that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

We need this divide to be random, so we'll use the `DataFrame.sample()`-method for this purpose:

In [4]:
randomized = spam_sms.sample(frac = 1, random_state = 1)

cutoff_index = round(len(randomized) * 0.8)

training = spam_sms[:cutoff_index].reset_index(drop=True)

testing = spam_sms[cutoff_index:].reset_index(drop=True)

Let's look at the percentage of spam in the respective sets and see if they appear representative:

In [5]:
print(training['Label'].value_counts(normalize=True) * 100, '\n')
print(testing['Label'].value_counts(normalize=True) * 100)

ham     86.496187
spam    13.503813
Name: Label, dtype: float64 

ham     86.983842
spam    13.016158
Name: Label, dtype: float64


They are both quite similar to our original percentages. This seems good enough for our purpose.

# Data cleaning
To make our classifications, we need to clean our data a bit so that it is in a more appropriate format.

## Punctuation and letter case
Firstly, we'll want to remove all punctuation from the messages and turn them all into lower case. The regular expression '\W' detect any character that is not from a-z, A-Z or 0-9. We'll utilize this to replace such characters.

In [6]:
training['SMS'] = training['SMS'].str.replace('\W', ' ')
training['SMS'] = training['SMS'].str.lower()
training.head()

,Label,SMS
0,ham,go until jurong point crazy available only ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i don t think he goes to usf he lives aro...


## Creating the vocabulary
Next, we will compile all words used in a vocabulary (a set of all unique words used):

In [7]:
training['SMS'] = training['SMS'].str.split()

word_set = set()
for message in training['SMS']:
    for word in message:
        word_set.add(word)

vocab = list(word_set)

It appears we have 7813 unique words in our vocabulary.

In [8]:
len(vocab)

7813

## The final training set
Lastly, we will transform the `SMS`-column into several columns, each representing a different word in the vocabulary. The values will represent how many occurences of that word we have in the corresponding messages.

To do this we first create a dictionary, `words_per_sms`, where each key corresponds to a word in the vocabulary and each value corresponds to a vector representing how many times that word was used in the respective rows.

In [9]:
word_counts_per_sms = {unique_word: [0] * len(training['SMS']) for unique_word in vocab}

for index, sms in enumerate(training['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

We then use this dictionary when creating a new dataframe:

In [10]:
word_counts = pd.DataFrame(word_counts_per_sms)

Lastly, we merge this dataframe with the `Label`-series, using the `pd.concat()`-function.

In [11]:
training_final = pd.concat([training, word_counts], axis = 1)
training_final.head()

,Label,SMS,sink,yrs,lonely,glory,fps,hg,mi,aig,...,slowly,condition,nigeria,flute,07,4742,haventcn,09061702893,anna,chennai
0,ham,"[go, until, jurong, point, crazy, available, o...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[ok, lar, joking, wif, u, oni]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,spam,"[free, entry, in, 2, a, wkly, comp, to, win, f...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,"[u, dun, say, so, early, hor, u, c, already, t...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[nah, i, don, t, think, he, goes, to, usf, he,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Calculations
The classification will be done comparing the following two equations:
![alt text](classification.png "Classification")
To calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we use the following equations (utilizing [additive smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) to avoid certain issues with 0-count words):
![alt text](calc.png "Conditional probability")
In this case alpha will be set to 1, that is, we'll be using *Laplace smoothing*.

Let's start doing the necessary calculations to make our classification possible.
## Constants
We will begin by calculating the constant values; P(Spam) and P(Ham), followed by NSpam, NHam and NVocabulary.
Lastly we'll initialize a variable named alpha with a value of 1 (in accordance with the forementioned laplace smoothing).

In [12]:
p_spam = (training_final['Label'] == 'spam').sum() / training_final.shape[0]

p_ham = 1 - p_spam

N_spam = training_final.loc[training_final['Label'] == 'spam', 'SMS'].apply(len).sum() #The number of words in all spam messages

N_ham  = training_final.loc[training_final['Label'] == 'ham', 'SMS'].apply(len).sum() #The number of words in all non-spam messages

N_vocab = len(vocab) #Total number of unique words

alpha = 1

## Parameters
The probability values of P(wi|Spam) and P(wi|ham) will be different depending on the word. These area called parameters and it is what we'll be calculating next. A key point is that these will remain constant as long as our training set doesn't change.

Recall that Nwi|Spam is equal to the number of times the word wi occurs in all the spam messages, while Nwi|Ham is equal to the number of times the word wi occurs in all the ham messages.

The fact that all of these will be pre-calculated before any classification is done, contributes to making the algorithm very fast.

In [13]:
# Initializing two dictionaries a key for every unique word, each with the initial value 0
p_cond_spam = dict.fromkeys(vocab, 0)
p_cond_ham = dict.fromkeys(vocab, 0)

spam_messages = training_final[training_final['Label'] == 'spam']
ham_messages = training_final[training_final['Label'] == 'ham']

for word in vocab:
    p_cond_spam[word] = (spam_messages[word].sum() + alpha) / (N_spam + N_vocab * alpha)
    p_cond_ham[word] = (ham_messages[word].sum() + alpha) / (N_ham + N_vocab * alpha)

# Classifying a new message
With parameters and constants calculated, we can now start creating the spam filter. It can be understood according to below:
- Takes in as input a new message (w1, w2, ..., wn)
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
    - If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
    - If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
    - If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.
    
For every incoming message we'll do a similar cleaning like before:
- Remove punctuation
- Transform to lower case
- Split string to list

In [14]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in p_cond_spam:
            p_spam_given_message *= p_cond_spam[word]
            
        if word in p_cond_ham:
            p_ham_given_message *= p_cond_ham[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

With that funtion done, let's try it out with two messages, one of which obviously is spam and one that is not.

In [15]:
classify("WINNER!! This is the secret code to unlock the money: C3421.")

P(Spam|message): 1.1946675407236375e-25
P(Ham|message): 2.5383900524029554e-27
Label: Spam


In [16]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 6.307218028475593e-25
P(Ham|message): 4.417049039139304e-21
Label: Ham


The function outputs the desired result for our two test examples. So far so good!

## Measuring the spam filter's accuracy
Next, we want to look at the testing set we compiled in the beginning, to see with what accuracy we can succeed to classify the messages as spam and non-spam. First off, we'll alter our previous `classify`-function to return the classification instead of printing it. We can now use the function to create a new column.

In [17]:
def classify_test_set(message):
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in p_cond_spam:
            p_spam_given_message *= p_cond_spam[word]
            
        if word in p_cond_ham:
            p_ham_given_message *= p_cond_ham[word]
            
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

To measure accuracy we'll divide the number of correctly classified messages with the total number of messages.

In [23]:
testing['predicted'] = testing['SMS'].apply(classify_test_set)

correct = 0
total = testing.shape[0]

for index, row in testing.iterrows():
    if classify_test_set(row['SMS']) == row['Label']:
        correct += 1

accuracy = correct / total
print("Accuracy:", accuracy)

Accuracy: 0.9856373429084381


We managed to achieve an accuracy of around 98.6%. This is way above our goal of 80% accuracy.